<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>

# Continuous training pipeline with KFP and Cloud AI Platform

In this lab, you will review, deploy, and run a KFP pipeline that orchestrates **BigQuery** and **Cloud AI Platform** services to train a **scikit-learn** model.




## Understanding the pipeline design
The pipeline source code can be found in the `pipeline` folder.

In [1]:
!ls -la pipeline

total 20
drwxr-xr-x 2 root root 4096 Mar 12 03:45 .
drwxr-xr-x 6 root root 4096 Mar 12 03:45 ..
-rw-r--r-- 1 root root 7260 Mar 12 03:34 covertype_training_pipeline.py
-rw-r--r-- 1 root root 2846 Mar 12 03:14 helper_components.py


The workflow implemented by the pipeline is defined using a Python based KFP Domain Specific Language (DSL). The pipeline's DSL is in the `covertype_training_pipeline.py` file. 

The pipeline's DSL has been designed to avoid hardcoding any environment specific settings like file paths or connection strings. These settings are provided to the pipeline code through a set of environment variables.


In [2]:
!grep 'BASE_IMAGE =' -A 5 pipeline/covertype_training_pipeline.py

BASE_IMAGE = os.getenv('BASE_IMAGE')
TRAINER_IMAGE = os.getenv('TRAINER_IMAGE')
RUNTIME_VERSION = os.getenv('RUNTIME_VERSION')
PYTHON_VERSION = os.getenv('PYTHON_VERSION')
COMPONENT_URL_SEARCH_PREFIX = os.getenv('COMPONENT_URL_SEARCH_PREFIX')



The pipeline uses a mix of custom and pre-build components.

- Pre-build components. The pipeline uses the following pre-build components that are included with the KFP distribution:
    - [BigQuery query component](https://github.com/kubeflow/pipelines/tree/0.2.4/components/gcp/bigquery/query)
    - [AI Platform Training component](https://github.com/kubeflow/pipelines/tree/0.2.4/components/gcp/ml_engine/train)
    - [AI Platform Deploy component](https://github.com/kubeflow/pipelines/tree/0.2.4/components/gcp/ml_engine/deploy)
- Custom components. The pipeline uses two custom helper components that encapsulate functionality not available in any of the pre-build components. The components are implemented using the KFP SDK's [Lightweight Python Components](https://www.kubeflow.org/docs/pipelines/sdk/lightweight-python-components/) mechanism. The code for the components is in the `helper_components.py` file:
    - **Retrieve Best Run**. This component retrieves the tuning metric and hyperparameter values for the best run of the AI Platform Training hyperparameter tuning job.
    - **Evaluate Model**. This component evaluates the *sklearn* trained model using a provided metric and a testing dataset.
    
The custom components execute in a container image defined in `base_image/Dockerfile`.

In [5]:
!cat base_image/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire scikit-learn==0.20.4 pandas==0.24.2 kfp==0.2.5


The training step in the pipeline employes the AI Platform Training component to schedule a  AI Platform Training job in a custom training container. The custom training image is defined in `trainer_image/Dockerfile`.

In [4]:
!cat trainer_image/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]


## Building and deploying the pipeline

Before deploying to AI Platform Pipelines, the pipeline DSL has to be compiled into a pipeline runtime format, also refered to as a pipeline package.  The runtime format is based on [Argo Workflow](https://github.com/argoproj/argo), which is expressed in YAML. 


### Configure environment settings

Update  the below constants  with the settings reflecting your lab environment. 

- `REGION` - the compute region for AI Platform Training and Prediction
- `ARTIFACT_STORE` - the GCS bucket created during installation of AI Platform Pipelines. The bucket name starts with the `hostedkfp-default-` prefix.
- `ENDPOINT` - set the `ENDPOINT` constant to the endpoint to your AI Platform Pipelines instance. Then endpoint to the AI Platform Pipelines instance can be found on the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console.

1. Open the *SETTINGS* for your instance
2. Use the value of the `host` variable in the *Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SKD* section of the *SETTINGS* window.

In [33]:
REGION = 'us-central1'
ENDPOINT = '11980e931804006b-dot-us-central2.pipelines.googleusercontent.com'
ARTIFACT_STORE_URI = 'gs://hostedkfp-default-1yu1vvsv25'

### Build the trainer image

In [12]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

IMAGE_NAME='trainer_image'
TAG='latest'
TRAINER_IMAGE='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

TRAINER_IMAGE

'gcr.io/mlops-dev-100/trainer_image:latest'

In [13]:
!gcloud builds submit --timeout 15m --tag $TRAINER_IMAGE trainer_image

Creating temporary tarball archive of 3 file(s) totalling 3.6 KiB before compression.
Uploading tarball of [trainer_image] to [gs://mlops-dev-100_cloudbuild/source/1583986299.41-d5db5947a0ee408889678c33db2eeea0.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/mlops-dev-100/builds/85201496-a5cc-48e8-aee4-9be69eb20349].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/85201496-a5cc-48e8-aee4-9be69eb20349?project=286479790129].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "85201496-a5cc-48e8-aee4-9be69eb20349"

FETCHSOURCE
Fetching storage object: gs://mlops-dev-100_cloudbuild/source/1583986299.41-d5db5947a0ee408889678c33db2eeea0.tgz#1583986299674523
Copying gs://mlops-dev-100_cloudbuild/source/1583986299.41-d5db5947a0ee408889678c33db2eeea0.tgz#1583986299674523...
/ [1 files][  1.7 KiB/  1.7 KiB]                                                
Operation completed over 1 objects/1.7 KiB.                

### Build the base image for custom components

In [16]:
IMAGE_NAME='base_image'
TAG='latest'
BASE_IMAGE=TRAINER_IMAGE='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [17]:
!gcloud builds submit --timeout 15m --tag $BASE_IMAGE base_image

Creating temporary tarball archive of 2 file(s) totalling 244 bytes before compression.
Uploading tarball of [base_image] to [gs://mlops-dev-100_cloudbuild/source/1583986788.97-a2088edcb59947c1afe93dbc399388c3.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/mlops-dev-100/builds/1789cc50-88c4-4f8a-b422-88e558cba5f6].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/1789cc50-88c4-4f8a-b422-88e558cba5f6?project=286479790129].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "1789cc50-88c4-4f8a-b422-88e558cba5f6"

FETCHSOURCE
Fetching storage object: gs://mlops-dev-100_cloudbuild/source/1583986788.97-a2088edcb59947c1afe93dbc399388c3.tgz#1583986789269658
Copying gs://mlops-dev-100_cloudbuild/source/1583986788.97-a2088edcb59947c1afe93dbc399388c3.tgz#1583986789269658...
/ [1 files][  289.0 B/  289.0 B]                                                
Operation completed over 1 objects/289.0 B.                 

### Compile the pipeline

You can compile the DSL using an API from the **KFP SDK** or using the **KFP** compiler.

To compile the pipeline DSL using **KFP** compiler

In [34]:
COMPONENT_URL_SEARCH_PREFIX = 'https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/'
RUNTIME_VERSION = '2.1'
PYTHON_VERSION = '3.7'

%env BASE_IMAGE={BASE_IMAGE}
%env TRAINER_IMAGE={TRAINER_IMAGE}
%env COMPONENT_URL_SEARCH_PREFIX={COMPONENT_URL_SEARCH_PREFIX}
%env RUNTIME_VERSION={RUNTIME_VERSION}
%env PYTHON_VERSION={PYTHON_VERSION}

env: BASE_IMAGE=gcr.io/mlops-dev-100/base_image:latest
env: TRAINER_IMAGE=gcr.io/mlops-dev-100/base_image:latest
env: COMPONENT_URL_SEARCH_PREFIX=https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/
env: RUNTIME_VERSION=2.1
env: PYTHON_VERSION=3.7


In [35]:
!dsl-compile --py pipeline/covertype_training_pipeline.py --output covertype_training_pipeline.yaml

The result is the `covertype_training_pipeline.yaml` file. 

In [36]:
!head covertype_training_pipeline.yaml

"apiVersion": |-
  argoproj.io/v1alpha1
"kind": |-
  Workflow
"metadata":
  "annotations":
    "pipelines.kubeflow.org/pipeline_spec": |-
      {"description": "The pipeline training and deploying the Covertype classifierpipeline_yaml", "inputs": [{"name": "project_id"}, {"name": "region"}, {"name": "source_table_name"}, {"name": "gcs_root"}, {"name": "dataset_id"}, {"name": "evaluation_metric_name"}, {"name": "evaluation_metric_threshold"}, {"name": "model_id"}, {"name": "version_id"}, {"name": "replace_existing_version"}, {"default": "\n{\n    \"hyperparameters\":  {\n        \"goal\": \"MAXIMIZE\",\n        \"maxTrials\": 6,\n        \"maxParallelTrials\": 3,\n        \"hyperparameterMetricTag\": \"accuracy\",\n        \"enableTrialEarlyStopping\": True,\n        \"params\": [\n            {\n                \"parameterName\": \"max_iter\",\n                \"type\": \"DISCRETE\",\n                \"discreteValues\": [500, 1000]\n            },\n            {\n                \"para

### Deploy the pipeline package

In [38]:
PIPELINE_NAME='covertype_classifier_training'

!kfp --endpoint $ENDPOINT pipeline upload \
-p $PIPELINE_NAME \
covertype_training_pipeline.yaml

Pipeline cd5d6afe-fcdf-4df6-b879-9efd430456a7 has been submitted

Pipeline Details
------------------
ID           cd5d6afe-fcdf-4df6-b879-9efd430456a7
Name         covertype_classifier_training
Description
Uploaded at  2020-03-12T04:51:15+00:00
+-----------------------------+--------------------------------------------------+
| Parameter Name              | Default Value                                    |
+=============================+==================================================+
| project_id                  |                                                  |
+-----------------------------+--------------------------------------------------+
| region                      |                                                  |
+-----------------------------+--------------------------------------------------+
| source_table_name           |                                                  |
+-----------------------------+--------------------------------------------------+
| gcs_r

## Submitting pipeline runs

You can trigger pipeline runs using an API from the KFP SDK or using KFP CLI. To submit the run using KFP CLI, execute the following commands. Notice how the pipeline's parameters are passed to the pipeline run.

### List the pipelines in AI Platform Pipelines

In [39]:
!kfp --endpoint $ENDPOINT pipeline list

+--------------------------------------+-------------------------------------------------+---------------------------+
| Pipeline ID                          | Name                                            | Uploaded at               |
+======================================+=================================================+===========================+
| cd5d6afe-fcdf-4df6-b879-9efd430456a7 | covertype_classifier_training                   | 2020-03-12T04:51:15+00:00 |
+--------------------------------------+-------------------------------------------------+---------------------------+
| b29a6dcf-4dda-4320-881d-9d8b3a762211 | [Tutorial] DSL - Control structures             | 2020-03-12T02:31:33+00:00 |
+--------------------------------------+-------------------------------------------------+---------------------------+
| 18691881-cba5-413b-907f-59b6a9f8d901 | [Tutorial] Data passing in python components    | 2020-03-12T02:31:32+00:00 |
+--------------------------------------+--------

Find the ID of the `covertype_classifier_training` pipeline you uploaded in the previous step and update the value of `PIPELINE_ID` .

In [40]:
PIPELINE_ID='cd5d6afe-fcdf-4df6-b879-9efd430456a7'

### Submit a run

In [52]:
EXPERIMENT_NAME = 'Covertype_Classifier_Training'
RUN_ID = 'Run_001'
SOURCE_TABLE = 'covertype_dataset.covertype'
DATASET_ID = 'splits'
EVALUATION_METRIC = 'accuracy'
EVALUATION_METRIC_THRESHOLD = '0.69'
MODEL_ID = 'covertype_classifier'
VERSION_ID = 'v01'
REPLACE_EXISTING_VERSION = 'True'

GCS_STAGING_PATH = '{}/staging'.format(ARTIFACT_STORE_URI)

In [53]:
!kfp --endpoint $ENDPOINT run submit \
-e $EXPERIMENT_NAME \
-r $RUN_ID \
-p $PIPELINE_ID \
project_id=$PROJECT_ID \
gcs_root=$GCS_STAGING_PATH \
region=$REGION \
source_table_name=$SOURCE_TABLE \
dataset_id=$DATASET_ID \
evaluation_metric_name=$EVALUATION_METRIC \
evaluation_metric_threshold=$EVALUATION_METRIC_THRESHOLD \
model_id=$MODEL_ID \
version_id=$VERSION_ID \
replace_existing_version=$REPLACE_EXISTING_VERSION

Creating experiment Covertype_Classifier_Training.
Run 3f2f73e1-d7ac-4618-a006-ee8e244dbfba is submitted
+--------------------------------------+---------+----------+---------------------------+
| run id                               | name    | status   | created at                |
+======================================+=========+==========+===========================+
| 3f2f73e1-d7ac-4618-a006-ee8e244dbfba | Run_001 |          | 2020-03-12T05:05:15+00:00 |
+--------------------------------------+---------+----------+---------------------------+


where

- EXPERIMENT_NAME is set to the experiment used to run the pipeline. You can choose any name you want. If the experiment does not exist it will be created by the command
- RUN_ID is the name of the run. You can use an arbitrary name
- PIPELINE_ID is the id of your pipeline. Use the value retrieved by the   `kfp pipeline list` command
- GCS_STAGING_PATH is the URI to the GCS location used by the pipeline to store intermediate files. By default, it is set to the `staging` folder in your artifact store.
- REGION is a compute region for AI Platform Training and Prediction. 

You should be already familiar with these and other parameters passed to the command. If not go back and review the pipeline code.


### Monitoring the run

You can monitor the run using KFP UI. Follow the instructor who will walk you through the KFP UI and monitoring techniques.

To access the KFP UI in your environment use the following URI:

https://[ENDPOINT]


*Note that your pipeline may fail due to the bug in a BigQuery component that does not handle certain race conditions. If you observe the pipeline failure retry the run from the KFP UI*
